In [1]:
import os
import sys
import cv2
import pytesseract
from PIL import Image, ImageDraw,ImageFont
from pytesseract import Output

In [2]:
nb_dir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

import config
import src.utilities.app_context as app_context
app_context.init()

#path to craft model, weight can be pulled form the production bracnh of repo
config.CRAFT_MODEL_PATH= nb_dir + '/utilities/craft_pytorch/model/craft_mlt_25k.pth'
config.CRAFT_REFINE_MODEL_PATH = nb_dir + '/utilities/craft_pytorch/model/craft_refiner_CTW1500.pth'
config.LINE_LAYOUT_MODEL_PATH= nb_dir + '/utilities/primalinenet/anuvaad_line_v1.pth'
config.LINE_LAYOUT_CONFIG_PATH = nb_dir + '/utilities/primalinenet/mask_rcnn_R_50_FPN_3x_v1.yaml'

In [3]:
from src.services.main import TextDetection

[2021-08-13 11:27:50,056] {__init__.py:241} MainThread DEBUG in __init__: (private) matplotlib data path: /home/naresh/ds-env2/lib/python3.6/site-packages/matplotlib/mpl-data
[2021-08-13 11:27:50,057] {__init__.py:241} MainThread DEBUG in __init__: matplotlib data path: /home/naresh/ds-env2/lib/python3.6/site-packages/matplotlib/mpl-data
[2021-08-13 11:27:50,062] {__init__.py:241} MainThread DEBUG in __init__: CONFIGDIR=/home/naresh/.config/matplotlib
[2021-08-13 11:27:50,065] {__init__.py:1480} MainThread DEBUG in __init__: matplotlib version 3.3.4
[2021-08-13 11:27:50,071] {__init__.py:1481} MainThread DEBUG in __init__: interactive is False
[2021-08-13 11:27:50,074] {__init__.py:1482} MainThread DEBUG in __init__: platform is linux
[2021-08-13 11:27:50,075] {__init__.py:1483} MainThread DEBUG in __init__: loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encoding

[2021-08-13 11:27:50,148] {__init__.py:241} MainThread DEBUG in __init__: CACHEDIR=/home/naresh/.cache/matplotlib
[2021-08-13 11:27:50,157] {font_manager.py:1436} MainThread DEBUG in font_manager: Using fontManager instance from /home/naresh/.cache/matplotlib/fontlist-v330.json
[2021-08-13 11:27:50,327] {pyplot.py:290} MainThread DEBUG in pyplot: Loaded backend module://ipykernel.pylab.backend_inline version unknown.
[2021-08-13 11:27:50,407] {geos.py:73} MainThread DEBUG in geos: Found GEOS DLL: <CDLL '/home/naresh/ds-env2/lib/python3.6/site-packages/shapely/.libs/libgeos_c-a68605fd.so.1.13.1', handle 421f860 at 0x7f7aaa4c0518>, using it.
[2021-08-13 11:27:50,419] {geos.py:38} MainThread DEBUG in geos: Trying `CDLL(libc.so.6)`
[2021-08-13 11:27:50,420] {geos.py:55} MainThread DEBUG in geos: Library path: 'libc.so.6'
[2021-08-13 11:27:50,421] {geos.py:56} MainThread DEBUG in geos: DLL: <CDLL 'libc.so.6', handle 7f7b3ebf6000 at 0x7f7aaa4c0780>


IndexError: tuple index out of range

[2021-08-13 11:27:51,191] {pyplot.py:290} MainThread DEBUG in pyplot: Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [ ]:
#base_dir = '/home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input'
base_dir = '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/'
filename    = 'hi_const255_hi.pdf'

#filename    = 'hamlet_1.pdf'
file_format = 'pdf'
language    =  'hi'

In [ ]:
app_context.application_context = {
  "input":{
  "inputs": [
    {
      "file": {
        "identifier": "string",
        "name": filename,
        "type": file_format
      },
      "config": {
        "OCR": {
          "option": "HIGH_ACCURACY",
          "language": "hi"
        }
      }
    }
  ]}
}


In [ ]:
config.LANGUAGE_LINE_THRESOLDS['hi']['text_threshold']=0.5
config.LANGUAGE_LINE_THRESOLDS['hi']['link_threshold']=0.35
config.LANGUAGE_LINE_THRESOLDS['hi']['low_text']=0.5

In [ ]:
resp = TextDetection(app_context,base_dir)

In [ ]:
file_index = 0
page_index =0
filepath = resp['rsp']['outputs'][file_index]['page_info'][page_index]

save_dir = '/home/naresh/word_detector_benchmark/'
#resp['rsp']['outputs'][file_index]['pages'][page_index]['words'][0]

In [ ]:

def draw_box(resp,filepath,save_dir, thresh,color="green", save=False):
    image  = Image.open(filepath)
    draw   = ImageDraw.Draw(image)
    for i in resp['rsp']['outputs'][file_index]['pages'][page_index]['words']:
        #font = ImageFont.truetype("sans-serif.ttf", 30)
        draw.text((10, 10),thresh,(255,0,255))
        draw.rectangle(((i['boundingBox']['vertices'][0]['x'], i['boundingBox']['vertices'][0]['y']), (i['boundingBox']['vertices'][2]['x'],i['boundingBox']['vertices'][2]['y'])), outline=color,width=3)
        
    
    save_filepath = os.path.join(save_dir, "bbox_"+str(thresh)+os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

In [ ]:
#local draw box
def draw_box(resp,filepath,save_dir,color="green", save=False):
    image  = Image.open(filepath)
    draw   = ImageDraw.Draw(image)
    for i in resp['rsp']['outputs'][file_index]['pages'][page_index]['lines']:
        draw.rectangle(((i['boundingBox']['vertices'][0]['x'], i['boundingBox']['vertices'][0]['y']), (i['boundingBox']['vertices'][2]['x'],i['boundingBox']['vertices'][2]['y'])), outline=color,width=3)

    save_filepath = os.path.join(save_dir, "bbox_"+os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

In [ ]:
image =draw_box(resp,filepath,save_dir,color="green", save=True)       

In [ ]:
def draw_tess(filepath,save_dir):
    img = cv2.imread(filepath)
    name = filepath.split("/")[-1]
    print(name)
    h, w, c = img.shape
    boxes = pytesseract.image_to_boxes(img)
    #try:
    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(d['text'])
    for i in range(n_boxes):
        if int(d['conf'][i]) > 1:
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imwrite(save_dir+name, img)
    print(save_dir+name)
#     except:
#         pass
    

In [ ]:
#draw_tess(filepath,save_dir)